# Problem 1:

LeetCode 966: https://leetcode.com/problems/vowel-spellchecker/

This is a simplified version for spell correction, since it only checks two common types of spell errors. Spell correction is widely used in many cases, like search engine and word processing.

## Description

Given a wordlist, we want to implement a spellchecker that converts a query word into a correct word.

For a given query word, the spell checker handles two categories of spelling mistakes:

- Capitalization: If the query matches a word in the wordlist (case-insensitive), then the query word is returned with the same case as the case in the wordlist.

- Vowel Errors: If after replacing the vowels ('a', 'e', 'i', 'o', 'u') of the query word with any vowel individually, it matches a word in the wordlist (case-insensitive), then the query word is returned with the same case as the match in the wordlist.

## My solution

In [9]:
import collections
import math

def find_word(word_dict: collections.defaultdict(list), word: str):
    """
    find if a word match any word in word dictionary

    Return:
        the exact match if exists
        first match otherwise
    """       
    final_word = ""

    word_lowcase = word.lower()
    if word_lowcase in word_dict:
        # grab all word candiates
        word_candidate_tuples = word_dict[word_lowcase]
        word_candidates = [word_tuple[0] for word_tuple in word_candidate_tuples]

        if word in word_candidates:
            # find the exact match
            final_word = word
        else:
            # find the first match
            final_word = word_candidates[0]

    return final_word
    
def generate_new_words(word: str) -> set:
    """
    generate all new words by replacing vowels in a word
    """

    vowel_list = ['a', 'e', 'i', 'o', 'u']

    word_lowcase = word.lower()

    # creat a set holding new word set 
    new_word_set = set()
    new_word_set.add(word_lowcase)

    for i in range(len(word_lowcase)):
        curr_char = word_lowcase[i]

        if curr_char not in vowel_list:
            continue

        for vowel in vowel_list:

            # generate new words by replacing the current letter with 
            # every vowel for every word in the new word set
            curr_word_set = new_word_set.copy()                
            for w in curr_word_set:
                new_word = w[:i] + vowel+ w[i+1:]
                new_word_set.add(new_word)

    return new_word_set

def find_first_match(word_set: set, word_dict):
    """
    given a set of words, find the matching word with smallest index
    """

    final_word = ''

    matching_word_lst = []       
    for w in word_set:
        if w in word_dict:
            matching_word_lst += word_dict[w]
    # print('matching_word_lst:', matching_word_lst) 

    idx_min = math.inf
    for word_tuple in matching_word_lst:
        w = word_tuple[0]
        idx = word_tuple[1]

        if idx < idx_min:
            idx_min = idx
            final_word = w

    return final_word


def spellchecker(wordlist: list, queries: list) -> list:  
    """
    Given a wordlist, converts a query word into a correct word.
    """

    # create a dictionary:
    # key: lower case of words in wordlist
    # value: list of tuples: (original words whose lower case matches key, index)
    word_dict = collections.defaultdict(list)

    for i in range(len(wordlist)):
        word = wordlist[i]
        word_dict[word.lower()].append((word, i))      
    # print(word_dict)

    res = []
    for word in queries:     
        # print(word, "***")
        final_word = ""
        
        # check if word without subtition appears in worddict:
        final_word = find_word(word_dict, word)

        if final_word:
            res.append(final_word)
            continue

        # check if substitue a vowl would find a match
        new_words = generate_new_words(word)
        final_word = find_first_match(new_words, word_dict)

        res.append(final_word)

    return res


In [10]:
wordlist = ["KiTe","kite","hare","Hare"]
queries = ["kite","Kite","KiTe","Hare","HARE","Hear","hear","keti","keet","keto"]
expect_output = ["kite","KiTe","KiTe","Hare","hare","","","KiTe","","KiTe"]

spellchecker(wordlist, queries)

['kite', 'KiTe', 'KiTe', 'Hare', 'hare', '', '', 'KiTe', '', 'KiTe']

In [11]:
wordlist = ["ae","aa"]
queries = ["UU"]
expect_output = ["ae"]

spellchecker(wordlist, queries)

['ae']

## Leetcode solution

We analyze the 3 cases that the algorithm needs to consider: when the query is an exact match, when the query is a match up to capitalization, and when the query is a match up to vowel errors.

In all 3 cases, we can use a hash table to query the answer.

- For the first case (exact match), we hold a set of words to efficiently test whether our query is in the set.
- For the second case (capitalization), we hold a hash table that converts the word from its lowercase version to the original word (with correct capitalization).
- For the third case (vowel replacement), we hold a hash table that converts the word from its lowercase version with the vowels masked out, to the original word.

In [ ]:
def spellchecker(self, wordlist, queries):
    def devowel(word):
        return "".join('*' if c in 'aeiou' else c
                       for c in word)

    words_perfect = set(wordlist)
    words_cap = {}
    words_vow = {}

    # create three dictionaries
    for word in wordlist:
        wordlow = word.lower()
        
        # only hash the first matching instance in the wordlist
        # so we will return the first word that satisfies the condition 
        words_cap.setdefault(wordlow, word) 
        words_vow.setdefault(devowel(wordlow), word)  

    def solve(query):
        if query in words_perfect:
            return query

        queryL = query.lower()
        if queryL in words_cap:
            return words_cap[queryL]

        queryLV = devowel(queryL)
        if queryLV in words_vow:
            return words_vow[queryLV]
        return ""

    return map(solve, queries)


- map() function returns a map object(which is an iterator) of the results after applying the given function to each item of a given iterable (list, tuple etc.)
    
- In Dictionary, setdefault() method returns the value of a key (if the key is in dictionary). If not, it inserts key with a value to the dictionary.

# Problem 2

LeetCode 72: Edit Distance (https://leetcode.com/problems/edit-distance/)

This problem describes the case where you want to find the correction word with minimum changes to the misspelled word.

## Description

Given two words word1 and word2, find the minimum number of operations required to convert word1 to word2.

You have the following 3 operations permitted on a word:

- Insert a character
- Delete a character
- Replace a character